# Deploying custom istio resources

## install kubernetes python library

In [1]:
!pip install kubernetes

## Imports

In [2]:
from kubernetes import client, config
import yaml

## Yaml Templates

In [3]:
envoy_filter_yaml = """
apiVersion: networking.istio.io/v1alpha3
kind: EnvoyFilter
metadata:
  name: add-header
  namespace: kubeflow-user
spec:
  configPatches:
  - applyTo: VIRTUAL_HOST
    match:
      context: SIDECAR_OUTBOUND
      routeConfiguration:
        vhost:
          name: dask-scheduler.kubeflow-user.svc.cluster.local:8786
          route:
            name: default
    patch:
      operation: MERGE
      value:
        request_headers_to_add:
        - append: true
          header:
            key: kubeflow-userid
            value: kubeflow-user
"""


virtual_service_yaml = """                
apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: dask-scheduler
  namespace: kubeflow-user
spec:
  gateways:
  - kubeflow/kubeflow-gateway
  hosts:
  - '*'
  http:
  - match:
    - uri:
        prefix: /apps/kubeflow-user/dask/
    rewrite:
      uri: /
    route:
    - destination:
        host: dask-scheduler-ui.kubeflow-user.svc.cluster.local
        port:
          number: 80
    timeout: 300s
"""

## Setup to access k8s

In [4]:
# get in-cluster config
config.load_incluster_config()

# Client API Interface
api_client = client.ApiClient()

# V1 Core API Interface
v1_api = client.CoreV1Api()

# interface to manipulate custom resources
api_custom_object = client.CustomObjectsApi(api_client)

## Deploy the istio custom resources

In [5]:
# convert yaml template to dictionary to pass in api call
envoy_filter = yaml.load(envoy_filter_yaml, Loader=yaml.SafeLoader)
virtual_service = yaml.load(virtual_service_yaml, Loader=yaml.SafeLoader)

In [6]:
# deploy custom objects
virtual_service_object = api_custom_object.create_namespaced_custom_object(
    group='networking.istio.io', 
    version='v1alpha3',
    namespace='kubeflow-user',
    plural='virtualservices',
    body=virtual_service
)

envoy_filter_object = api_custom_object.create_namespaced_custom_object(
    group='networking.istio.io', 
    version='v1alpha3',
    namespace='kubeflow-user',
    plural='envoyfilters',
    body=envoy_filter
)
ef_object = api_custom_object.get_namespaced_custom_object(
    group='networking.istio.io', 
    version='v1alpha3',
    namespace='kubeflow-user',
    plural='envoyfilters',
    name='add-header'
)

## Read objects

In [7]:
ef_object

{'apiVersion': 'networking.istio.io/v1alpha3',
 'kind': 'EnvoyFilter',
 'metadata': {'creationTimestamp': '2021-12-24T16:22:50Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'networking.istio.io/v1alpha3',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {}, 'f:configPatches': {}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2021-12-24T16:22:50Z'}],
  'name': 'add-header',
  'namespace': 'kubeflow-user',
  'resourceVersion': '51412',
  'uid': '7c629087-0cd7-4ba5-b2f3-8d0003d74753'},
 'spec': {'configPatches': [{'applyTo': 'VIRTUAL_HOST',
    'match': {'context': 'SIDECAR_OUTBOUND',
     'routeConfiguration': {'vhost': {'name': 'dask-scheduler.kubeflow-user.svc.cluster.local:8786',
       'route': {'name': 'default'}}}},
    'patch': {'operation': 'MERGE',
     'value': {'request_headers_to_add': [{'append': True,
        'header': {'key': 'kubeflow-userid', 'value': 'kubeflow-user'}}]}}}]}}

In [8]:
vs_object = api_custom_object.get_namespaced_custom_object(
    group='networking.istio.io', 
    version='v1alpha3',
    namespace='kubeflow-user',
    plural='virtualservices',
    name='dask-scheduler'
)
vs_object

{'apiVersion': 'networking.istio.io/v1alpha3',
 'kind': 'VirtualService',
 'metadata': {'creationTimestamp': '2021-12-24T16:22:50Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'networking.istio.io/v1alpha3',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {},
      'f:gateways': {},
      'f:hosts': {},
      'f:http': {}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2021-12-24T16:22:50Z'}],
  'name': 'dask-scheduler',
  'namespace': 'kubeflow-user',
  'resourceVersion': '51410',
  'uid': 'fe26151c-8752-4590-9d16-60fbbd79ce23'},
 'spec': {'gateways': ['kubeflow/kubeflow-gateway'],
  'hosts': ['*'],
  'http': [{'match': [{'uri': {'prefix': '/apps/kubeflow-user/dask/'}}],
    'rewrite': {'uri': '/'},
    'route': [{'destination': {'host': 'dask-scheduler-ui.kubeflow-user.svc.cluster.local',
       'port': {'number': 80}}}],
    'timeout': '300s'}]}}

## Show custom objects

In [9]:
!kubectl get vs

NAME                                     GATEWAYS                        HOSTS   AGE
notebook-kubeflow-user-k8sapi-notebook   ["kubeflow/kubeflow-gateway"]   ["*"]   23m
dask-scheduler                           ["kubeflow/kubeflow-gateway"]   ["*"]   1s


In [10]:
!kubectl get envoyfilters

NAME         AGE
add-header   1s


## Clean up

In [11]:
# delete the custom objects
result = api_custom_object.delete_namespaced_custom_object(
    group='networking.istio.io', 
    version='v1alpha3',
    namespace='kubeflow-user',
    plural='virtualservices',
    name='dask-scheduler'
)

result = api_custom_object.delete_namespaced_custom_object(
    group='networking.istio.io', 
    version='v1alpha3',
    namespace='kubeflow-user',
    plural='envoyfilters',
    name='add-header'
)

## Confirm removal of custom objects

In [12]:
!kubectl get virtualservices

NAME                                     GATEWAYS                        HOSTS   AGE
notebook-kubeflow-user-k8sapi-notebook   ["kubeflow/kubeflow-gateway"]   ["*"]   23m


In [13]:
!kubectl get envoyfilters

No resources found in kubeflow-user namespace.


## Help Doc on Custom Object Manipulation

In [14]:
help(api_custom_object.create_namespaced_custom_object)

Help on method create_namespaced_custom_object in module kubernetes.client.api.custom_objects_api:

create_namespaced_custom_object(group, version, namespace, plural, body, **kwargs) method of kubernetes.client.api.custom_objects_api.CustomObjectsApi instance
    create_namespaced_custom_object  # noqa: E501
    
    Creates a namespace scoped Custom object  # noqa: E501
    This method makes a synchronous HTTP request by default. To make an
    asynchronous HTTP request, please pass async_req=True
    >>> thread = api.create_namespaced_custom_object(group, version, namespace, plural, body, async_req=True)
    >>> result = thread.get()
    
    :param async_req bool: execute request asynchronously
    :param str group: The custom resource's group name (required)
    :param str version: The custom resource's version (required)
    :param str namespace: The custom resource's namespace (required)
    :param str plural: The custom resource's plural name. For TPRs this would be lowercase pl

In [15]:
help(api_custom_object.delete_namespaced_custom_object)

Help on method delete_namespaced_custom_object in module kubernetes.client.api.custom_objects_api:

delete_namespaced_custom_object(group, version, namespace, plural, name, **kwargs) method of kubernetes.client.api.custom_objects_api.CustomObjectsApi instance
    delete_namespaced_custom_object  # noqa: E501
    
    Deletes the specified namespace scoped custom object  # noqa: E501
    This method makes a synchronous HTTP request by default. To make an
    asynchronous HTTP request, please pass async_req=True
    >>> thread = api.delete_namespaced_custom_object(group, version, namespace, plural, name, async_req=True)
    >>> result = thread.get()
    
    :param async_req bool: execute request asynchronously
    :param str group: the custom resource's group (required)
    :param str version: the custom resource's version (required)
    :param str namespace: The custom resource's namespace (required)
    :param str plural: the custom resource's plural name. For TPRs this would be lower